c:\Users\user\Desktop\data ia projet\ProjetMaladiesCardiaques\DATA


In [1]:
from knn_model import train_knn_model
from random_forest_model import train_random_forest_model
from regression_logistique_model import train_regression_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [2]:
df=pd.read_csv('donnée_malades_cardiaques.csv',index_col=0)

X = df.drop(columns=["Maladie_cardiaque"])  
Y = df["Maladie_cardiaque"]


In [3]:
# Charger les données
df = pd.read_csv('donnée_malades_cardiaques.csv', index_col=0)
X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement (retourne aussi le scaler)
rl, scaler = train_regression_model(X, Y)  # Utiliser le même scaler

# Prendre un échantillon de test
num_samples = 100
X_sample = X.sample(n=num_samples, random_state=42)
Y_sample = Y.loc[X_sample.index]

# Appliquer la **même normalisation** qu'à l'entraînement
X_sample_scaled = scaler.transform(X_sample)  # Ne pas recréer un scaler !

# Prédictions avec le même modèle et scaler
rl_preds = rl.predict(X_sample_scaled)
rl_probs = rl.predict_proba(X_sample_scaled)
print("Shape de X_train :", X.shape)  # Doit être (N, 16)
print("Shape de X_sample_scaled :", X_sample_scaled.shape)  # Doit être (100, 16)


Shape de X avant la séparation :  (917, 16)
Shape de X_train après séparation :  (733, 16)
Shape de X_test après séparation :  (184, 16)
Shape de X_train_scaled après normalisation :  (733, 16)
Shape de X_test_scaled après normalisation :  (184, 16)
Meilleur modèle trouvé :  LogisticRegression(C=0.01, max_iter=1000, solver='liblinear')
Shape de X_train : (917, 16)
Shape de X_sample_scaled : (100, 16)


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
def generate_comparison_table(X, Y, knn, rf, rl, scaler, num_samples=100):
    # Sélectionner aléatoirement des échantillons
    X_sample = X.sample(n=num_samples, random_state=42)
    Y_sample = Y.loc[X_sample.index]
    
    # Vérifiez que X_sample a le bon nombre de caractéristiques avant la normalisation
    print(f"Shape de X_sample avant transformation : {X_sample.shape}")

    # Normalisation des données
    X_scaled = scaler.transform(X_sample)
    
    # Vérifiez la forme de X_scaled après transformation
    print(f"Shape de X_scaled après normalisation : {X_scaled.shape}")
    
    # Si le modèle LogisticRegression (rl) a été formé avec 10 caractéristiques, appliquez une sélection
    # ou transformation ici pour que X_scaled ait également 10 caractéristiques avant de faire la prédiction.
    
    # Prédictions des modèles
    knn_preds = knn.predict(X_scaled)
    knn_probs = knn.predict_proba(X_scaled)
    
    rf_preds = rf.predict(X_scaled)
    rf_probs = rf.predict_proba(X_scaled)
    
    
    rl_preds = rl.predict(X_scaled)
    rl_probs = rl.predict_proba(X_scaled)
    
    # Construction du DataFrame de comparaison
    comparison_df = X_sample.copy()
    comparison_df['Vraie_classe'] = Y_sample.values
    
    comparison_df['KNN_Prediction'] = knn_preds
    comparison_df['KNN_Prob_Sain'] = knn_probs[:, 0] * 100
    comparison_df['KNN_Prob_Malade'] = knn_probs[:, 1] * 100
    
    comparison_df['RF_Prediction'] = rf_preds
    comparison_df['RF_Prob_Sain'] = rf_probs[:, 0] * 100
    comparison_df['RF_Prob_Malade'] = rf_probs[:, 1] * 100
    
    comparison_df['RL_Prediction'] = rl_preds
    comparison_df['RL_Prob_Sain'] = rl_probs[:, 0] * 100
    comparison_df['RL_Prob_Malade'] = rl_probs[:, 1] * 100
    
    return comparison_df


# Charger les données
df = pd.read_csv('donnée_malades_cardiaques.csv', index_col=0)
X = df.drop(columns=['Maladie_cardiaque'])  
Y = df['Maladie_cardiaque']

# Entraînement des modèles
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
knn = train_knn_model(X, Y)
rf = train_random_forest_model(X, Y)
rl, _ = train_regression_model(X, Y)

# Générer la comparaison
tableau_comparaison = generate_comparison_table(X, Y, knn, rf, rl, scaler)

# Afficher les premières lignes
tableau_comparaison.head()


Shape de X avant la séparation :  (917, 16)
Shape de X_train après séparation :  (733, 16)
Shape de X_test après séparation :  (184, 16)
Shape de X_train_scaled après normalisation :  (733, 16)
Shape de X_test_scaled après normalisation :  (184, 16)
Meilleur modèle trouvé :  LogisticRegression(C=0.01, max_iter=1000, solver='liblinear')
Shape de X_sample avant transformation : (100, 16)
Shape de X_scaled après normalisation : (100, 16)


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Âge,Sexe,Tension_aurepos,Cholestérol,Glycémieà_jeun,Fréquence_cardiaque_max,Angine_d_effort,Dépression_ST,Type_de_douleur_thoracique_ASY,Type_de_douleur_thoracique_ATA,...,Vraie_classe,KNN_Prediction,KNN_Prob_Sain,KNN_Prob_Malade,RF_Prediction,RF_Prob_Sain,RF_Prob_Malade,RL_Prediction,RL_Prob_Sain,RL_Prob_Malade
669,45,0,130,234,0,175,0,0,0,1,...,0,0,80.392157,19.607843,0,58.0,42.0,0,80.603178,19.396822
30,53,1,145,518,0,130,0,0,0,0,...,1,1,43.137255,56.862745,1,41.0,59.0,0,52.399530,47.600470
377,65,1,160,237,1,122,0,1,1,0,...,1,1,1.960784,98.039216,1,9.0,91.0,1,11.343574,88.656426
536,62,1,133,237,1,119,1,1,0,0,...,1,1,9.803922,90.196078,1,31.0,69.0,1,16.251317,83.748683
807,54,1,108,309,0,156,0,0,0,1,...,0,0,98.039216,1.960784,0,89.0,11.0,0,88.684712,11.315288
